In [1]:
import numpy as np
from numpy import linalg as la
from numpy import random
from scipy.stats import norm
from scipy.stats import genextreme
from scipy import optimize
from scipy.io import loadmat
from tabulate import tabulate
import NonLinearModels_ante as nlm

%load_ext autoreload
%autoreload 2

In this problem set, you will replicate part of the results in
Brownstone and Train (1999). You will estimate the conditional logit
model given the available data.

The Conditional Logit (CL) Model
================================

The Conditional Logit (CL) model is a powerful tool for analyzing
descrete choices, which is heavily used in e.g. empirical industrial
organization. In this model, decision makers have to choose between $J$
different discrete options, $\{1,2,...,J\}$. An option might be a car.
For each option, we observe a vector of characteristics,
$x_{j}\in\mathbb{R}^{K}$, and for each individual $i=1,...,N$, we
observe the chosen alternative, $y_{i}\in\{1,...,J\}$. If individuals
face different alternatives, e.g. if the prices or characteristics of
cars available were different, then characteristics also vary across
individuals, $x_{ij}\in\mathbb{R}^{K}$.

Our model assumes that individual $i$ chose the alternative that
maximized utility, 

$$y_{i}=\arg\max_{j\in\{1,...,J\}}u_{ij}.$$ 

Our model for utility takes the form

$$u_{ij}=x_{ij}\beta+\varepsilon_{ij},\quad\varepsilon_{ij}\sim\text{IID Extreme Value Type I}.$$

That is, utility is composed of a part that depends on observables,
$x_{ij}\beta$, and an idiosyncratic error term, $\varepsilon_{ij}$,
observed to the individual but not to the econometrician. The problem of
estimation is to recover $\beta$ without knowledge on
$\varepsilon_{ij}$.

It turns out that the distributional form for $\varepsilon_{ij}$ implies
that

$$
\Pr(y_{i}=j|\mathbf{X}_{i})=\frac{\exp(x_{ij}\beta)}{\sum_{k=1}^{J}\exp(x_{ik}\beta)},
$$

where $\mathbf{X}_{i}=(x_{i1},x_{i2},...,x_{iJ})$. This remarkable
result was first introduced to economists by nobel laureate Daniel
McFadden. Taking logarithms, we obtain a particularly parsimonious form
for the log-likelihood contribution:

$$\ell_{i}(\beta)=x_{ij}\beta-\log\sum_{k=1}^{J}\exp(x_{ik}\beta).$$


Max rescaling for numerical stability
-------------------------------------

A particularly important numerical trick that one is typically forced to
use with logit models is what is called *max rescaling*. For simplicity,
let $v_{ij}\equiv x_{ij}'\beta$. Now, note that for any
$K_{i}\in\mathbb{R}$, 

$$
\begin{aligned}
\frac{\exp(v_{ij})}{\sum_{k=1}^{J}\exp(v_{ik})} & = \frac{\exp(v_{ij})}{\sum_{k=1}^{J}\exp(v_{ik})}\frac{\exp(-K_{i})}{\exp(-K_{i})}\\
 & = \frac{\exp(v_{ij}-K_{i})}{\sum_{k=1}^{J}\exp(v_{ik}-K_{i})}.
 \end{aligned}
 $$

This means that we can subtract any scalar from all utilities for an
individual. This is very useful because the exponential function is a
highly unstable numerical object on any computer: for large or small
values of $z$, $\exp(z)$ will result in round-up or round-down errors,
respectively. Since round-up errors are particularly bad for estimation,
it turns out to be useful to choose $K_{i}$ so that we avoid them, even
at the cost of encountering more round-down errors. Thus, we choose

$$
K_{i}=\max_{j\in\{1,...,J\}}v_{ij},
$$ 

and subtract $K_{i}$ from all utilities before taking any exponential values.

Compensating variation in logit models
--------------------------------------

A useful feature of logit models is that they provide a neat welfare
measure in the form of what is commonly referred to as the "log sum."
This is because of the fact that

$$\mathbb{E}_{\varepsilon_{i1},...,\varepsilon_{iJ}}\left[\max(v_{ij}+\varepsilon_{ij})\right]=\log\left[\sum_{j=1}^{J}\exp(v_{ij})\right].$$

Because the left-hand side is the expected utility, prior to knowing the
error terms, of the choice instance, it can be thought of the "value" of
the choice instance. If one of the variables, say the first, is a price
variable, then $\beta_{1}$ is the marginal utility of price, converting
money into utils. Thus, economists tend to divide the welfare measure
with $\beta_{1}$ to get a money-metric utility measure. Again, the level
of that measure in itself may not be useful, but differences are.
Suppose we change something about the utilities, from $v_{ij}$ to
$\tilde{v}_{ij}$, and want to compute the compensating variation: that
is, how much we would have to pay the agent (regardless of the chosen
alternative) to make the agent indifferent between being placed in the
first or the second choice instance. We can compute that as

$$CV=\frac{1}{\beta_{1}}\log\left[\sum_{j=1}^{J}\exp(\tilde{v}_{ij})\right]-\frac{1}{\beta}\log\left[\sum_{j=1}^{J}\exp(v_{ij})\right].$$

If we add the monetary amount, $CV$, to all utilities in the baseline,
$v_{ij}$, then the expected maximum utility would be the same in the two
choice instances, and the agent would thus be indifferent between the
two.

Policy makers can use the $CV$ measure to compare how much better or
worse an individual is from a change in taxation, an introduction or
reduction in the choiceset, or a change in one or more of the attributes
of the alternatives.

Data
====

The data consists of a survey of households regarding their preferences
for car purchase. Each household was given 6 options, but the
characteristics that the respondents were asked about was varied. The
surveys were originally conducted in order to illicit consumer
preferences for alternative-fuel vehicles. The data is *stated
preferences*, in the sense that consumers did not actually buy but just
stated what they would hypothetically choose, which is of course a
drawback. This is very common in marketing when historic data is either
not available or does not provide satisfactory variation. The advantage
of the stated preference data is therefore that the choice set can be
varied greatly (for example, the characteristics includes the
availability of recharging stations, which is important for purchase of
electric cars).

The data you will use has $N=4654$ respondents with $J=6$ cars to choose
from. For each household, we let $\mathcal{J}_{i}$ denote the set of
available cars.

If you load the mat-file, `car_data.m`, you will get a dictionary with three variables: `y`, `x`, and `labels`. The variable `y` is $N\times1$ and
$\mathtt{y}_{i}\in\{1,2,3,4,5,6\}$ denotes the chosen car of household
$i$. The variable `x` is $N\times J\times K$, where $K=21$ is the number
of variables (their descriptions given in the table below. The
variable `labels` contains the names of the 21 variables.

|      | **Variable**            | **Description** |
|  ---- |------------------------| -------------------------------------------------------------------------------------------------------- |
|  1  |  Price/ln(income)       |  Purchase price in thousands of dollars, divided by the natural log of household income in thousands |
|  2  |  Range                  |  Hundreds of miles that the vehicle can travel between refuelings/rechargings |
|  3  |  Acceleration           |  Seconds required to reach 30 mph from stop, in tens of seconds (e.g., 3 s is entered as 0.3) |
|  4  |  Top speed              |  Highest speed that the vehicle can attain, in hundreds of miles/h (e.g., 80 mph is entered as 0.80) |
|  5  |  Pollution              |  Tailpipe emissions as fraction of comparable new gas vehicle |
|  6  |  Size                   |  0\"mini, 0.1\"subcompact, 0.2\"compact, 0.3\"mid-size or large |
|  7  |  1(Big enough)          |  1 if household size is over 2 and vehicle size is 3; 0 otherwise |
|  8  |  Luggage space          |  Luggage space as fraction of comparable new gas vehicle |
  |9  |  Operating cost          | Cost per mile of travel, in tens of cents per mile (e.g., 5 cents/mile is entered as 0.5.)
 | | |  For electric vehicles, cost is for home recharging. For other vehicles, cost is for station refueling |
|  10 |  Station availability   |  Fraction of stations that have capability to refuel/recharge the vehicle |
|  11 |  Sports utility vehicle |  1 for sports utility vehicle, zero otherwise |
|  12 |  Sports car             |  1 for sports car, zero otherwise |
|  13 |  Station wagon          |  1 for station wagon, zero otherwise |
|  14 |  Truck                  |  1 for truck, zero otherwise |
|  15 |  Van                    |  1 for van, zero otherwise |
|  16 |  Constant for EV        |  1 for electric vehicle, zero otherwise |
|  17 |  Commute\*EV            |  1 if respondent commutes less than five miles each day and vehicle is electric; zero otherwise |
|  18 |  College\*EV            |  1 if respondent had some college education and vehicle is electric; zero otherwise |
|  19 |  Constant for CNG       |  1 for compressed natural gas vehicle, zero otherwise |
|  20 |  Constant for methanol  |  1 for methanol vehicle, zero otherwise |
|  21 |  College\*methanol      |  1 if respondent had some college education and vehicle is methanol; zero otherwise |

3-dimensional arrays in Numpy
------------------------------

The explanatory variables are $x_{ijk}$, where $i$ denotes individuals,
$j$ cars, and $k$ car attributes. We have loaded the package `from scipy.io import loadmat`, read its documentation on how we can neatly import `y`, `x` and `labels`. Since Python is 0-indexed, it makes life easier if you take substract 1 from y.

Which of the following commands shows all the price-to-log-income
    values for the cars in the choiceset of individual 1 (and what does
    the other commands give you?).

    a:   x[:,1,1]

    b:   x[1,:,1]

    b:   x[1,1,:]

In [2]:
mat_data = loadmat('car_data.mat', squeeze_me=True)

In [3]:
y = mat_data.get('y')
y -= 1
x = mat_data.get('x')
x_lab = list(mat_data.get('labels'))

In [38]:
# Try to index, and make yourself familiar with a three dimensional array.

Next, we want to compute $\sum_{k=1}^{K}x_{ijk}\theta_{k}$ using linear algebra. Our  `x` matrix has three dimensions, $N\times J\times K$, and $\theta$ is $K\times1$. Does this behave as expected when using numpy?

### Question 2

Create a 3-dimensional matrix `A` that has dimensions $10 \times 4 \times 3$, and fill it with random draws from a normal distribution. What happens if you matrix multiply this by the $3 \times 1$ vector `1 2 3`? From "common" linear algebra knowledge, what do you expect the dimension of the new matrix is?

In [ ]:
rng = random.default_rng(seed=42)
A =  # Create the matrix specified in the question.
# Perform some matrix multiplication, does it look like expected?

Finish the clogit code
-----------------------

**Question 3.** Finish `simulate_dataset(N,J,theta)` and simulate a dataset. It must return `x` with dimensions ($N\times J\times K$) and `y` (or `u`, these are the same, but sometimes I mix the notation) with dimensions ($N\times1$).

To iterate, we are looking at the data generating process in the form of,
$$u_{ij} = x_{ij}\beta + \varepsilon_{ij},\quad\varepsilon_{ij}\sim\text{IID Extreme Value Type I}.$$

*Programming hint 1:* <br>
Now, $x_{ij}\beta + \varepsilon_{ij}$ has dimensions ($N\times J$), which represents the different utilities person $i$ gets from choice $j$. Since person $i$ will choose the option $j$ that have the highest utility, we need for each row get the index of the column that maximises utility (we want the index because we are interested in the choice that maximises utility, but not interested in that utility by itself). In numpy you can use the method `argmax` to get the column number that gives max, look up its documentation on how to use it.

*Programming hint 2:* <br>
You can draw Extreme Value errors using `genextreme.ppf(q, c)`. Where q is an array of likelihoods (could be some random draws from a uniform distribution), and c specifies which extreme distribution we should draw from. Setting `c=0` gives the extreme value I distribution, which we want to use.

In [7]:
n = 1000
k = 3
j = 4
thetaTrue = np.array([1, 2, 3])

In [8]:
# Finish the nlm.sim_data function.
sim_data = nlm.sim_data(n, j, thetaTrue)

**Question 4.** Finish `choice_prob(theta, x)`. It should return both the `ccp` matrix and the `logccp` matrix.

To reiterate, the choice probabilities are given by the function,
$$
\Pr(y_{i}=j|\mathbf{X}_{i})=\frac{\exp(x_{ij}\beta)}{\sum_{k=1}^{J}\exp(x_{ik}\beta)},
$$
Make sure you reread the section about max rescaling for numerical stability, or else you might have issues later when we do maximum likelihood.

In [9]:
# Finish the nlm.choice_prob function, I have made some checks that you can use to make sure you made the function correct.
ccp, logccp = nlm.choice_prob(np.zeros(k), sim_data['x'])
print((ccp == 0.25).all())
print(np.isclose(logccp, -1.38629436).all())

True
True


**Question 5.** Finish the `clogit(theta, y, x)` function. Now that you have written all of these function, you can simulate a fake
dataset and check that you can estimate the true parameters back using
`estimate()`.

*Hint:* Use the function `choice_prob` to get the $N\times J$ matrix of choice probabilities, `logccp`. You now need to use y vector to get the correct car (in other words, the correct column) that the person choose, and get the choice probability for that car. The nlm.clogit function should return a $N \times 1$ vector.

In [10]:
sim_results = nlm.estimate(
    nlm.clogit, np.zeros((k, )), sim_data['y'], sim_data['x']
)

Optimization terminated successfully.
         Current function value: 0.460613
         Iterations: 12
         Function evaluations: 52
         Gradient evaluations: 13


In [11]:
nlm.print_table(('y', ['theta 1 = 1.0', 'theta 2 = 2.0', 'theta 3 = 3.0']), sim_results)

Results
Dependent variable: y

                   Beta        Se    t-values
-------------  --------  --------  ----------
theta 1 = 1.0  0.961137  0.076385     12.5828
theta 2 = 2.0  2.0167    0.108805     18.535
theta 3 = 3.0  3.18816   0.158442     20.1219
In 12 iterations and 52 function evaluations.


Estimation on real data
-----------------------
**Question 6.** Estimate the 21 parameters of the model. Check that you are able to
    find the same estimates and standard errors as in Brownstone and
    Train (1998; p. 121).



In [12]:
bt_res = [-.185 , .350 , -.716 , .261 , -.444 , .935 , .143 , .501 , -.768 , .413 , .820 , .637 , -1.437 , -1.017 , -.799 , -.179 , .198 , .443 , .345 , .313 , .228]

In [13]:
n, j, k = x.shape
theta0 = np.zeros((k, ))

In [58]:
clogit_results = nlm.estimate(nlm.clogit, theta0, y, x)
nlm.print_table(('Car chosen', x_lab), clogit_results, floatfmt='.3f')

Optimization terminated successfully.
         Current function value: 1.588275
         Iterations: 86
         Function evaluations: 1914
         Gradient evaluations: 87
Results
Dependent variable: Car chosen

                          Beta     Se    t-values
----------------------  ------  -----  ----------
Price/ln(income)        -0.185  0.027      -6.817
Range                    0.350  0.027      12.999
Acceleration            -0.716  0.111      -6.479
Top speed                0.261  0.080       3.273
Pollution               -0.444  0.100      -4.436
Size                     0.934  0.311       3.004
Big enough               0.143  0.076       1.888
Luggage space            0.502  0.188       2.668
Operating cost          -0.768  0.073     -10.466
Station availability     0.413  0.097       4.278
Sports utility vehicle   0.819  0.144       5.680
Sports car               0.636  0.156       4.068
Station wagon           -1.437  0.065     -21.962
Truck                   -1.017  0.05

In [36]:
# Check that results we get is close to the ones in paper.
np.isclose(clogit_results['b_hat'], bt_res, rtol=0.01)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

## Price elasticities
As with standard binary probit or logit, the parameter estimates are not
interesting in and of themselves. Instead, we want to compute
*elasticities* that we can interpret and answer interesting questions
with. To do this, recall that an elasticity takes the form

$$\mbox{elasticity}=\frac{\mathrm{dy}}{y}\frac{x}{\mathrm{d}x}.$$ 
This
formula is useful for computing elasticities numerically: we can
increase $x$ by some small amount, say $10^{-5}$, and measure the change
in $y$. The elasticity is then the relative change in $y$ divided by the
relative change in $x$: $\frac{\Delta y/y}{\Delta x/x}$, or equivalently

$$\text{numerical elasticity}=\frac{\text{pct. change in }y}{\text{pct. change in }x}.$$

**Question 7.** Compute own-price elasticity of each car, $j=1,...,6$.

Here we use the formula

$$
El_{price}(p_{ij}) = \frac{price_j}{p_{ij}(x_{ij}, price_j)} \cdot \frac{p_{ij}(x_{ij}, price_j \cdot h) - p_{ij}(x_{ij}, price_j)}{(1 + h)\cdot price_j}
$$

where $x_{ij}$ are all the other regressors except for price, and $h = 1 + \varepsilon$, where $\varepsilon$ is a very small number, such as $1e-5$

Suggestion on how to implement this:
- Do these steps by looping over the cars in the data set. So you should create a loop from 0 to 6 (not including 6).
- First create a copy of $x$, let us call it $x2$ (make sure that you make a copy, and not a reference). In $x2$ multiply the price of the car (but for all households) by $h$.
- Get the choice probabilities using $x2$ and nlm.choice_prob (not the log choice probabilites).
- You should now be able to use the formula above to calculate the elasticity, and store it in `E_own`.
- When you have looped through all the cars, calculate the mean over all the values to get the average price elasticity.

*Note:* `E_own` has dimensions $N \times J$, so each row is the own price elasticity for some household, and each column hold the own price elasticity for each car.

In [25]:
# Original choice probabilites
ccp, _ = nlm.choice_prob(clogit_results['b_hat'], x)

In [27]:
E_own = np.zeros((n, j))
E_cross = np.zeros((n, j))
dpdx = np.zeros((n, j))

for car in range(j):
    # Copy
    x2 = x.copy()
    x2[:, car, 0] *= 1e-5

    ccp2, _ = nlm.choice_prob(clogit_results['b_hat'], x2)
    dccp = ccp2 - ccp

    x0 = x[:, car, 0]
    dx = x2[:, car, 0] - x0

    # Own price elasticity
    dy = dccp[:, car]
    y0 = ccp[:, car]
    E_own[:, car] = (x0/dx) * (dy/y0)

    # Cross price elasticity
    diff_car = [x != car for x in range(j)]
    dy = dccp[:, diff_car]
    y0 = ccp[:, diff_car]
    E_cross[:, car] = np.mean(dy/y0, axis=1) * (x0/dx)

In [454]:
print(np.mean(E_own))
print(np.mean(E_cross))

-0.6521698318958457
0.1278177603625842


**Question 8.** Compute the cross-price elasticities from each car $j=1,...,6$ on the remaining cars, $k\ne j$.

You perform the same loop as previous question, but instead use only the choice probabilities for all other cars, except for the current car in your loop. You can create a bollean to index the choice probabilites using the function,

    diff_car = [x != car for x in range(j)]

**Question 9.** Consider the electrical vehicles (EVs) in the dataset. What can you
    conclude about the demand for EVs compared to traditional cars? What
    does that imply for car producers? And for environmental policy?

In [457]:
# Create two indexed, from where idx1 is for electric cars
# and idx0 is for non-electric cars.
idx1 = x[:, :, 16]==1; 
idx0 = x[:, :, 16]==0; 
print(np.mean(E_own[idx1]))
print(np.mean(E_own[idx0]))

-0.7216596026003235
-0.6427609949256127
